# DataSet Import and Analysis

In this notebook, we are going to train an RNN for the task of sentiment analysis. Our dataset is the Yelp review data set. Let's download this data set and analyze it.

In [1]:
!pip install contractions
!pip install torch_geometric
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu118.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.9 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.8/887.8 kB 50.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

import contractions
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from torch.nn.utils.rnn import pad_sequence

import torch
from tqdm import tqdm
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.nn.norm import BatchNorm
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from sklearn.model_selection import train_test_split

#Weighting Module
import torch.nn.functional as F
import torch
import torch.nn as nn

import torch_cluster
from torch_geometric.nn import global_mean_pool, knn_graph

from sklearn.metrics import f1_score, roc_auc_score

from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
!gdown --id 1Ns456aZINhGcmnI_SCIHgT0q2x14jM2s

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Ns456aZINhGcmnI_SCIHgT0q2x14jM2s
To: /content/yelp.csv
100% 8.09M/8.09M [00:00<00:00, 42.9MB/s]


In [4]:
!gdown --id 1Xl92YAhgQVS2fR0MDUTJHcpN5_hxP67-

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Xl92YAhgQVS2fR0MDUTJHcpN5_hxP67-
To: /content/test-gpt.csv
100% 24.6k/24.6k [00:00<00:00, 53.2MB/s]


In [5]:
df = pd.read_csv('yelp.csv')
df.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  10000 non-null  object
 1   date         10000 non-null  object
 2   review_id    10000 non-null  object
 3   stars        10000 non-null  int64 
 4   text         10000 non-null  object
 5   type         10000 non-null  object
 6   user_id      10000 non-null  object
 7   cool         10000 non-null  int64 
 8   useful       10000 non-null  int64 
 9   funny        10000 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 781.4+ KB


In [7]:
df.describe()

,stars,cool,useful,funny
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,3.777500,0.876800,1.409300,0.701300
std,1.214636,2.067861,2.336647,1.907942
min,1.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,1.000000,0.000000
75%,5.000000,1.000000,2.000000,1.000000
max,5.000000,77.000000,76.000000,57.000000


In [8]:
# Now let's see how many samples there exists in this data set
print("Total number of samples is: {}".format(len(df)))
print("Total number of 1 star reviews is: {} and the fraction is:{:0.2f}".format(len(df[df['stars']==1]), len(df[df['stars']==1])/len(df)))
print("Total number of 2 star reviews is: {} and the fraction is:{:0.2f}".format(len(df[df['stars']==2]), len(df[df['stars']==2])/len(df)))
print("Total number of 3 star reviews is: {} and the fraction is:{:0.2f}".format(len(df[df['stars']==3]), len(df[df['stars']==3])/len(df)))
print("Total number of 4 star reviews is: {} and the fraction is:{:0.2f}".format(len(df[df['stars']==4]), len(df[df['stars']==4])/len(df)))
print("Total number of 5 star reviews is: {} and the fraction is:{:0.2f}".format(len(df[df['stars']==5]), len(df[df['stars']==5])/len(df)))

Total number of samples is: 10000
Total number of 1 star reviews is: 749 and the fraction is:0.07
Total number of 2 star reviews is: 927 and the fraction is:0.09
Total number of 3 star reviews is: 1461 and the fraction is:0.15
Total number of 4 star reviews is: 3526 and the fraction is:0.35
Total number of 5 star reviews is: 3337 and the fraction is:0.33


In [9]:
# We just need these rows: text, stars
df_new = df[['text', 'stars']].copy()
df_new.head()


,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


In [10]:
df_test = pd.read_csv('test-gpt.csv')
df_test.drop(columns='Unnamed: 0', inplace=True)
df_test.head()

,ratings,labels
0,The food was absolutely delicious! I couldn't ...,1
1,Excellent service and a charming ambiance. I h...,1
2,This restaurant has a fantastic selection of d...,1
3,The staff is incredibly attentive and friendly...,1
4,"I love how they use fresh, high-quality ingred...",1


# Text Preprocessing

Now, it's the time to apply text preprocessing. In text preprocessing we remove stopwords, remove punctuations, expand contractions, and make all words to lower.

In [11]:
stop_words = set(stopwords.words('english'))

In [12]:
print(stop_words)

{'didn', 'we', 'doesn', 'other', "you're", 'so', 'its', 'by', 'once', 'nor', 'him', 'on', 'she', "weren't", 'weren', "shan't", 'what', 'no', 'these', 'his', 'were', 'll', "shouldn't", 'after', 'o', 'those', 'hers', 'below', 'then', 'but', 'off', 'where', 'there', 'your', 'them', 'our', "mightn't", 'that', 'their', 'having', 'will', 'you', 'couldn', 'it', "don't", 'only', 'theirs', 'does', 'of', 'above', 'they', 'with', 've', 'a', 'out', 'own', 'hadn', "mustn't", "that'll", 'an', 'up', "wouldn't", 'did', 'ours', 'just', 'be', 'has', "won't", 'while', "it's", 'myself', 'themselves', 'itself', 'needn', 'some', 'being', 'mightn', 'don', 'shouldn', 'through', "haven't", "she's", "should've", 'i', 'not', 'me', 'to', 'shan', 'before', "hadn't", "you'll", 'was', 'is', 'or', 'my', 'wouldn', 'yourselves', 'd', 'any', 'yourself', 'herself', 'about', 'aren', 'too', 'doing', 'how', 'this', 'hasn', "couldn't", 'most', 'ma', 'isn', 'ourselves', 'himself', 're', "aren't", "wasn't", 'very', 'do', 'must

In [13]:
def expand_sent(sent):
  expanded_words = [ contractions.fix(word) for word in sent.split()]
  expanded_sent = " ".join(expanded_words)
  return expanded_sent

In [14]:
# Lets see an example
sent = "I'm not very satisfied. It isn't good!"
expand_sent(sent)

'I am not very satisfied. It is not good!'

In [15]:
def clean_sent(sent):
  sent = expand_sent(sent)
  tokens = word_tokenize(sent)
  tokens = [word.lower() for word in tokens]
  # tokens =[word for word in tokens if not (word in stop_words)]
  tokens = [word for word in tokens if word.isalpha()]
  return " ". join(tokens)

In [16]:
#Lets see an example
sent = "I'm not very satisfied. The price is 12$, it's not so cheep!!!???"
clean_sent(sent)


'i am not very satisfied the price is it is not so cheep'

In [17]:
# Now lets clean the sentences in the data set
from tqdm import tqdm
tqdm.pandas()
df_new['clean_text'] = df_new['text'].progress_apply(clean_sent)
# df_new['new_stars'] = df_new['stars'].apply(lambda star: star-1)
ratings={1:0, 2:0, 3:1, 4:1, 5:1}
df_new['new_stars'] = df_new['stars'].apply(lambda star: ratings[star])


100%|██████████| 10000/10000 [00:25<00:00, 387.83it/s]


In [18]:
tqdm.pandas()
df_test['ratings'] = df_test['ratings'].progress_apply(clean_sent)
df_test

100%|██████████| 300/300 [00:00<00:00, 5667.21it/s]


,ratings,labels
0,the food was absolutely delicious i could not ...,1
1,excellent service and a charming ambiance i ha...,1
2,this restaurant has a fantastic selection of d...,1
3,the staff is incredibly attentive and friendly...,1
4,i love how they use fresh ingredients the flav...,1
...,...,...
295,the staff was unattentive and it was hard to g...,0
296,the desserts were too sweet and it was overwhe...,0
297,the food lacked creativity and tasted like gen...,0
298,the restaurant had an insect problem which was...,0


# ‌Build Vocabulary for the data set

In this section, we write a function to create a vocabulary for a given data set. To this aim, each word should be mapped to unique index.

In [19]:
def build_vocab(df, min_freq):
  word2index ={}
  index2word = {}
  word_count ={}
  index2word[0] = '<PAD>'
  index2word[1] = '<UNK>'

  word2index['<PAD>'] = 0
  word2index['<UNK>'] = 1
  sentences = df['clean_text'].to_list()
  for sent in sentences:
    tokens = sent.split()
    for token in tokens:
      if  not (token in word_count):
        word_count[token] =1
      else:
        word_count[token]+=1
  index =2
  for word in word_count:
    if word_count[word]>=min_freq:
      index2word[index] = word
      word2index[word] = index
      index+=1
  return word2index, index2word, word_count

In [20]:
df_train, df_val = train_test_split(df_new, test_size=0.3, shuffle=True, random_state=123)

In [21]:
train_labels = [label for label in df_train['new_stars']]
val_labels = [label for label in df_val['new_stars']]
test_labels = [label for label in df_test['labels']]

In [22]:
min_freq = 4
word2index, index2word, word_count = build_vocab(df_train, min_freq)

In [23]:
# print(word2index)
len(word2index)

8604

In [24]:
# print(index2word)
len(index2word)

8604

In [25]:
train_tokens=[]
for sent in df_train['clean_text']:
  train_tokens.append([token for token in sent.split() if token in word2index])

In [26]:
val_tokens=[]
for sent in df_val['clean_text']:
  val_tokens.append([token for token in sent.split() if token in word2index])

In [27]:
test_tokens=[]
for sent in df_test['ratings']:
  test_tokens.append([token for token in sent.split() if token in word2index])

In [28]:
train_dict_list={}
def sentence2graph(sent):
  start, end = 0,4 # window length

  occurence = {} # occurence dict: node id: neighbors in window
  feature_matrix =[] #feature matrix
  i2w = {} # word to index dict in window-> word: id
  word_seq=[]
  sent_seq =[]
  i=0
  # constrict w2i and feature matrix
  for word in sent:
    i2w[i] = word
    word_seq.append(word2index[word])
    sent_seq.append(i)
    #feature_matrix.append(embedding_model.wv[word])
    i+=1
  #initializw occurence
  word_seq = torch.tensor(word_seq, dtype=torch.long)
  #feature_matrix = embedding(word_seq)
  for item in sent_seq:
    occurence[item]=set()
  #construct occurence
  while end<=len(sent_seq)+1:
    window = sent_seq[start:end]
    for word in window:
      for innerword in window:
        occurence[word].add(innerword)
    start+=1
    end+=1
  #construct edge_index
  edge_index_1 = []
  edge_index_2 = []
  for source in occurence:
    for dest in occurence[source]:
      edge_index_1.append(source)
      edge_index_2.append(dest)
  edge_index = torch.tensor([edge_index_1, edge_index_2], dtype=torch.long)
  sent_seq = torch.tensor(sent_seq, dtype=torch.long)
  return edge_index, word_seq, sent_seq #torch.tensor(feature_matrix, dtype=torch.float)



In [29]:
edge_index, word_seq, sent_seq = sentence2graph(train_tokens[3000])

Now, we have to write a mapping function that maps sentences to their indexes based on the vocabulary that we created.
We alse define a reversed mapping function that maps a sequence of indexes to the correponding words.

In [30]:
train_graphs =[]
val_graphs = []
for item, label in tqdm(zip(train_tokens, train_labels)):
  edge_index, word_seq, sent_seq = sentence2graph(item)
  train_graphs.append(Data(x=sent_seq, edge_index=edge_index,word_seq=word_seq,
                           y=torch.tensor(label, dtype=torch.long)))
for item, label in tqdm(zip(val_tokens, val_labels)):
  edge_index , word_seq, sent_seq = sentence2graph(item)
  val_graphs.append(Data(x=sent_seq, edge_index=edge_index,word_seq=word_seq
                          , y=torch.tensor(label, dtype=torch.long)))

7000it [00:06, 1132.84it/s]
3000it [00:01, 1618.63it/s]


In [31]:
test_graphs = []
for item, label in tqdm(zip(test_tokens, test_labels)):
  edge_index , word_seq, sent_seq = sentence2graph(item)
  test_graphs.append(Data(x=sent_seq, edge_index=edge_index,word_seq=word_seq
                          , y=torch.tensor(label, dtype=torch.long)))

300it [00:00, 8168.60it/s]


# Creating Pytorch Data Set


Now, we have to create a pytoch data set and also the data loaders

In [32]:
batch_size = 32
train_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_graphs, batch_size=batch_size, shuffle=False)

train_eval_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=True)
val_eval_loader = DataLoader(val_graphs, batch_size=batch_size, shuffle=False)

In [33]:
test_loader = DataLoader(test_graphs, batch_size=batch_size, shuffle=False)

test_eval_loader = DataLoader(test_graphs, batch_size=batch_size, shuffle=False)


# Weighting Module

In [34]:
class Weighting(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(Weighting, self).__init__()
    self.network = nn.Sequential(nn.Linear(input_dim, hidden_dim),
                                 nn.ReLU(),
                                 nn.Linear(hidden_dim, hidden_dim),
                                 nn.ReLU(),
                                 nn.Linear(hidden_dim, output_dim))
  def forward(self, x):
    # x: [batch_size, hidden_dim]
    x = self.network(x)
    x = torch.sigmoid(x)
    x = F.normalize(x, p=1, dim=0)

    return x


#The GCN Network

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
class GCNClassifier(nn.Module):
  def __init__(self, n_vocabs, embed_dim, hidden_dim, gcn_dim, n_classes, n_layers, bidirectional=True):
    super(GCNClassifier, self).__init__()
    self.Embedding = nn.Embedding(n_vocabs, embed_dim)
    self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True,
                       bidirectional=bidirectional,num_layers=n_layers,
                       dropout = 0.3)

    self.fc = nn.Linear(gcn_dim, n_classes)

    self.GCN1 = SAGEConv(hidden_dim*2 if bidirectional else hidden_dim, gcn_dim)
    self.batchnorm1 = BatchNorm(gcn_dim)

    self.GCN2 = SAGEConv(gcn_dim, gcn_dim)
    self.batchnorm2 = BatchNorm(gcn_dim)
    self.GCN3 = SAGEConv(gcn_dim, gcn_dim)
    self.batchnorm3 = BatchNorm(gcn_dim)

    self.relu = nn.ReLU()
    self.drop = nn.Dropout(0.5)

    self.padding_value= 0

  def rnn_prepare(self, graph_data, padding_value ):
      batch2nodes ={}
      batch_size = graph_data.y.shape[0]
      for i in range(batch_size):
        batch2nodes[i] = []
      for node, batch in zip(graph_data.word_seq, graph_data.batch):
        batch2nodes[batch.item()].append(node.item())
      seq_lists = [torch.tensor(batch2nodes[item], dtype=torch.long) for item in batch2nodes]
      seq_lens = [len(batch2nodes[item]) for item in batch2nodes]
      seq_lists_padded = pad_sequence(seq_lists, batch_first=True, padding_value=padding_value)
      return seq_lists_padded, seq_lens

  def forward(self, data):
    # print(data)
    seq_lists_padded, seq_lens = self.rnn_prepare(graph_data = data, padding_value= self.padding_value)
    seq_lists_padded = seq_lists_padded.to(device)
    #seq: [batch_size, seq_len]
    seq_features = self.Embedding(seq_lists_padded)
    seq_features= self.drop(seq_features)
    # x = self.drop(x)
    #x: [batch_size, seq_len, embed_dim]
    lstm_features, _ = self.rnn(seq_features)
    # rnn_out, (rnn_h, rnn_c) = self.rnn(x)
    x_h = torch.cat([lstm_features[i,0:item,:] for i, item in enumerate(seq_lens)], 0)
    edge_index = data.edge_index
    x = self.GCN1(x_h, edge_index)
    x = self.batchnorm1(x)
    x = self.relu(x)
    x = self.GCN2(x, edge_index)
    x = self.batchnorm2(x)
    x = self.relu(x)
    # x = self.GCN3(x, edge_index)
    # x = self.batchnorm3(x)
    # x = self.relu(x)
    x_h = global_mean_pool(x, data.batch, size=data.y.shape[0])
    x = self.drop(x_h)
    x = self.fc(x)

    return F.log_softmax(x, dim=-1), x_h

In [37]:
def encode_onehot_torch(num_classes, labels):
    y = torch.eye(num_classes).to(device)
    return y[labels]


In [38]:
def loss_function_classifier(outputs, labels, weights, num_classes):
  onehot_labels = encode_onehot_torch(num_classes, labels)
  return torch.sum(- weights * (onehot_labels.float() * outputs).sum(1), -1)


In [39]:
def loss_function_weights(outputs, labels, weights, num_classes, gamma):
  onehot_labels = encode_onehot_torch(num_classes, labels)
  return torch.sum(- weights * (onehot_labels.float() * outputs).sum(1), -1) - gamma*torch.sum(weights*torch.log(weights+1e-20))


In [40]:
#Initializations
n_vocabs = len(word2index)
embed_dim = 64
hidden_dim = 64
gcn_dim=32
n_classes = 2
n_layers = 1
bidirectional= False
model_classifier = GCNClassifier(n_vocabs, embed_dim, hidden_dim, gcn_dim, n_classes, n_layers, bidirectional)
model_classifier = model_classifier.to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [41]:
w_inputdim = gcn_dim
w_hiddendim=32
weight_lr = 4e-3
model_weights = []
weight_optimizers=[]
for i in range(n_classes):
  model_weights.append(Weighting(w_inputdim, w_hiddendim, 1).to(device))
  weight_optimizers.append(torch.optim.Adam(model_weights[-1].parameters(), lr=weight_lr, ))

In [42]:
print("number of model parameters:{}".format(sum(p.numel() for p in model_classifier.parameters() if p.requires_grad)))

number of model parameters:592482


# Define Loss Function and Optimizer

In [43]:
classifier_lr = 2e-3
classifier_optimizer = torch.optim.Adam(model_classifier.parameters(), lr=classifier_lr)

# The Training and Test Loops

In [44]:
from tqdm import tqdm
epoch_classifier=3
epoch_weights = 3
gamma=1.0

In [45]:
def zero_grad_both():
  classifier_optimizer.zero_grad()
  for opt in weight_optimizers:
    opt.zero_grad()

In [46]:
def run_classifier(texts):
  output_probs, embds = model_classifier(texts)
  return output_probs, embds

In [47]:
def run_weights_generator(embds, labels):
  batch_size = embds.shape[0]
  max_label = int(labels.max().item() + 1)
  weights = torch.empty(batch_size).to(device)
  for i in range(max_label):
    labels_indices = (labels == i).nonzero().squeeze()
    if labels_indices is not None:
      sub_embds = embds[labels_indices,:]
      weight_ = model_weights[i](sub_embds)
      weights[labels_indices] = weight_.squeeze()
      # weights = weights / max_label
  return weights

In [48]:
def train_one_epoch():
  model_classifier.train()
  # result_dict={}
  for model in model_weights:
    model.train()

  for epoch_clf in range(epoch_classifier):
    print("Epoch {}/{} of classifier training".format(epoch_clf, epoch_classifier))
    loss_clfs = []
    acc_clfs = []
    for data in tqdm(train_loader):
      data = data.to(device)
      labels =data.y.to(device)
      zero_grad_both()
      output_probs, embds = run_classifier(data)
      weights1 = run_weights_generator(embds, labels)
      loss_clf = loss_function_classifier(output_probs, labels, weights1, n_classes)
      # loss_clf = criterion(output_probs, labels)
      loss_clf.backward()
      loss_clfs.append(loss_clf.item())
      preds = torch.argmax(output_probs,dim=-1)
      acc_clfs.append(sum(preds==labels)/len(labels))
      classifier_optimizer.step()

    loss_clfs_avg = sum(loss_clfs)/len(loss_clfs)
    acc_clfs_avg = sum(acc_clfs)/len(acc_clfs)
    print("classifier loss:{:.3f}, classifier acc:{:.3f}".format(loss_clfs_avg,acc_clfs_avg ))
  for epoch_w in range(epoch_weights):
    print("Epoch {}/{} of Weight generator training".format(epoch_w, epoch_weights))
    loss_Ws =[]
    acc_Ws =[]
    for data in tqdm(train_loader):
      data = data.to(device)
      labels = data.y.to(device)
      zero_grad_both()
      output_probs, embds = run_classifier(data)
      weights2 = run_weights_generator(embds, labels)
      loss_W = -loss_function_weights(output_probs, labels, weights2, n_classes, gamma)
      loss_Ws.append(loss_W.item())
      preds = torch.argmax(output_probs, dim=-1)
      acc_Ws.append(sum(preds==labels)/len(labels))
      loss_W.backward()
      for opt in weight_optimizers:
        opt.step()

    loss_W_avg = sum(loss_Ws)/len(loss_Ws)
    acc_W_avg = sum(acc_Ws)/len(acc_Ws)

    print("weight generator loss:{:.3f}, classifier acc:{:.3f}".format(loss_W_avg,acc_W_avg ))

  return

In [49]:
def class_f1(output, labels, type='micro', pos_label=None):
    # preds = output.max(1)[1].type_as(labels)
    preds = torch.argmax(output, -1)
    if pos_label is None:
        return f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average=type)
    else:
        return f1_score(labels.cpu().numpy(), preds.cpu(), average=type, pos_label=pos_label)


In [50]:
def evaluate_train():
  model_classifier.eval()
  loss = 0
  acc = 0
  outs = []
  lbls = []
  print('evaluating train')
  with torch.inference_mode():
    for data in tqdm(train_eval_loader):
      data = data.to(device)
      labels = data.y.to(device)
      output_probs, _ = run_classifier(data)
      loss = loss + F.nll_loss(output_probs, labels).item()
      preds = torch.argmax(output_probs, dim=-1)
      acc = acc+ (sum(preds==labels) / len(labels))
      outs.append(output_probs)
      lbls.append(labels)
      # f1_score = class_f1(output_probs, labels, type='macro')
    outs = torch.cat(outs)
    lbls = torch.cat(lbls)
    f1_score = class_f1(outs, lbls, type='macro')

  return loss/len(train_eval_loader), acc / len(train_eval_loader), f1_score


In [51]:
def evaluate_val():
  model_classifier.eval()
  loss = 0
  acc =0
  outs = []
  lbls = []
  print('evaluating validation')
  with torch.inference_mode():
    for data in tqdm(val_eval_loader):
      data = data.to(device)
      labels = data.y.to(device)
      output_probs, _ = run_classifier(data)
      loss = loss + F.nll_loss(output_probs, labels).item()
      preds = torch.argmax(output_probs, dim=-1)
      acc = acc+ (sum(preds==labels) / len(labels))
      outs.append(output_probs)
      lbls.append(labels)
      # f1_score = class_f1(output_probs, labels, type='macro')
    outs = torch.cat(outs)
    lbls = torch.cat(lbls)
    f1_score = class_f1(outs, lbls, type='macro')

  return loss / len(val_eval_loader), acc/len(val_eval_loader), f1_score

In [52]:
def evaluate_test():
  model_classifier.eval()
  loss = 0
  acc =0
  outs = []
  lbls = []
  print('evaluating test')
  with torch.inference_mode():
    for data in tqdm(test_eval_loader):
      data = data.to(device)
      labels = data.y.to(device)
      output_probs, _ = run_classifier(data)
      loss = loss + F.nll_loss(output_probs, labels).item()
      preds = torch.argmax(output_probs, dim=-1)
      acc = acc+ (sum(preds==labels) / len(labels))
      outs.append(output_probs)
      lbls.append(labels)
      # f1_score = class_f1(output_probs, labels, type='macro')
    outs = torch.cat(outs)
    lbls = torch.cat(lbls)
    f1_score = class_f1(outs, lbls, type='macro')

  return loss / len(test_eval_loader), acc/len(test_eval_loader), f1_score

In [53]:
epochs = 20
for epoch in range(epochs):
    train_one_epoch()
    train_loss, train_acc, train_f1_score = evaluate_train()
    val_loss, val_acc, val_f1_score = evaluate_val()
    test_loss, test_acc, test_f1_score = evaluate_test()
    print("Major Results for Epoch{}/{}".format(epoch, epochs))
    print("Train loss:{:.3f}, Train acc:{:.3f}, Train f1_score:{:.3f}".format(train_loss, train_acc, train_f1_score))
    print("Validation loss:{:.3f}, Validation acc: {:.3f}, Validation f1_score:{:.3f}".format(val_loss, val_acc, val_f1_score))
    print("Test loss:{:.3f}, Test acc: {:.3f}, Test f1_score:{:.3f}".format(test_loss, test_acc, test_f1_score))
    print("*********************************************")
    print("*********************************************")
    print("*********************************************")

Epoch 0/3 of classifier training


100%|██████████| 219/219 [00:30<00:00,  7.16it/s]


classifier loss:1.252, classifier acc:0.634
Epoch 1/3 of classifier training


100%|██████████| 219/219 [00:29<00:00,  7.52it/s]


classifier loss:1.010, classifier acc:0.728
Epoch 2/3 of classifier training


...

evaluating validation


100%|██████████| 94/94 [00:11<00:00,  8.22it/s]


evaluating test


100%|██████████| 10/10 [00:00<00:00, 68.03it/s]

Major Results for Epoch19/20
Train loss:0.006, Train acc:0.997, Train f1_score:0.995
Validation loss:0.741, Validation acc: 0.885, Validation f1_score:0.783
Test loss:0.973, Test acc: 0.799, Test f1_score:0.778
*********************************************
*********************************************
*********************************************


In [54]:
complete_loader = DataLoader(val_graphs, batch_size = 200)
target_names = ['Negative', 'Positive']
for data in complete_loader:
  model_classifier.eval()
  data, labels = data.to(device), data.y.to(device)
  logits, _ = model_classifier(data)
  preds = torch.argmax(logits, dim=-1)

labels, preds = labels.cpu().numpy(), preds.cpu().numpy()

print(classification_report(y_true=labels, y_pred=preds, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.71      0.62      0.67        32
    Positive       0.93      0.95      0.94       168

    accuracy                           0.90       200
   macro avg       0.82      0.79      0.80       200
weighted avg       0.90      0.90      0.90       200



In [55]:
complete_loader = DataLoader(test_graphs, batch_size = len(test_graphs))
target_names = ['Negative', 'Positive']
for data in complete_loader:
  model_classifier.eval()
  data, labels = data.to(device), data.y.to(device)
  logits, _ = model_classifier(data)
  preds = torch.argmax(logits, dim=-1)

labels, preds = labels.cpu().numpy(), preds.cpu().numpy()

print(classification_report(y_true=labels, y_pred=preds, target_names=target_names))

              precision    recall  f1-score   support

    Negative       0.67      0.76      0.71       100
    Positive       0.87      0.81      0.84       200

    accuracy                           0.80       300
   macro avg       0.77      0.79      0.78       300
weighted avg       0.81      0.80      0.80       300

